# <b> <u>Jammer Classification: </u> </b>
### Example Notebook Showing how you can train and save the modeles!
### Step1: Import All Dependencies!

In [1]:
# Install the dependencies from the requirements.txt

!pip install -q -r requirements.txt

# Import tensor flow and keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

# tensorflow hub is used to load the models
!pip install -q tensorflow_hub
import tensorflow_hub as hub

import numpy as np

# Import Matplot lib, Library to plot the results and graphs
import matplotlib.pylab as plt 

# Argument parser to set variables
import options

# import dataloader
from util import dataLoader

# import generate model module
from CNN import generate_model

### Step2: Setup all Required Variables

##### Delete sys.argv(req for ipython Kernels)

In [2]:
# Try to parse Argument from options.py
import sys; sys.argv=['']; del sys
args = options.parseArguments()

#Set the mode to train or usePreTrain
args.mode = "usePreTrain"
args.freeze_feature_layers = True

# Pre-Trained model to use
args.pre_trained_model_name = "inception_v3"
args.model_url = "https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(args.pre_trained_model_name)

# Set the Dataset Directory
args.input_dir = "/media/aadmin/SamsungSSD/PROJECT-DATA/SIGENCE/Train_Val_Test/dataset/"

# Set the image Dimensions and batch size according to the pre-trained model's requirement Check it on tf-hub
args.img_h = args.img_w = 299
args.batch_size = 100

# Set the load directory, where trained models and graphs are saved
args.save_dir = "./checkpoints/3_Classes/"



### Define the Function for Training


In [3]:
#use Pre Trained Models for Fine Tuning
def usePreTrain(args):
    # Generate Data Loader
    dataloader = dataLoader.DataLoader(args)
    train_data_gen, val_data_gen = dataloader.dataGenerator()
    
    #genrateModel
    generateModel = generate_model.Generate_model(args, dataloader.num_classes)
    model = generateModel.preTrainedModel()
    
    # Fit your Training Data
    history = model.fit(train_data_gen, epochs=2, verbose=2,
                steps_per_epoch= dataloader.num_train_images // args.batch_size)
    
    keras.experimental.export_saved_model(model, args.save_dir)
    

#### Start Training

In [4]:
if args.mode == "usePreTrain":
    usePreTrain(args)

elif args.mode=="train":
    train(args)
    

Initializing DataLoader ...
Looking for the data at --> /media/aadmin/SamsungSSD/PROJECT-DATA/SIGENCE/Train_Val_Test/dataset/
Classes : ['01_ChirpJammerSignals', '04_NarrowBandSignals'] -----------
Training Data Loader
Found 5000 images belonging to 2 classes.
Validation Data Loader
Found 400 images belonging to 2 classes.
Do Fine Tuning of feature layers :  False
Building model from -->  https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 2048)              21802784  
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 21,806,882
Trainable params: 4,098
Non-traina